Paper<br>
https://arxiv.org/abs/2209.14988v1<br>
<br>
GitHub<br>
https://github.com/ashawkey/stable-dreamfusion<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Prismer_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup environment

## git clone

In [ ]:
%cd /content

!git clone https://github.com/ashawkey/stable-dreamfusion.git

%cd /content/stable-dreamfusion
# Commits on Apr 23, 2023
!git checkout 4171f00c8d1721bb4645bad902b8b4d6fae3cef5

## install libraries

In [ ]:
%cd /content/stable-dreamfusion

# install requirements
! pip install -r requirements.txt
! pip install 'git+https://github.com/NVlabs/nvdiffrast/@335cfa6b33d785730a04283994214bed57884e87'

# install CUDA extensions (takes about 8 minutes!)
! pip install ./raymarching
! pip install ./shencoder
! pip install ./freqencoder
! pip install ./gridencoder

# install moviepy
!pip install moviepy imageio==2.4.1

## import libraries

In [ ]:
import os
import glob

import torch
torch.cuda.empty_cache()

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

# Setup prompt

In [ ]:
Prompt_text = "a DSLR photo of a delicious banana" #@param {type: 'string'}
Training_iters = 5000 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 12 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Max_steps = 512 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

#@markdown ---

Workspace = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}
Workspace_test = "trial" #@param{type: 'string'}

# processings
Prompt_text = "'" + Prompt_text + "'"

# Training

In [ ]:
%cd /content/stable-dreamfusion

!python main.py \
  -O \
  --text {Prompt_text} \
  --workspace {Workspace} \
  --iters {Training_iters} \
  --lr {Learning_rate} \
  --w {Training_nerf_resolution} \
  --h {Training_nerf_resolution} \
  --seed {Seed} \
  --lambda_entropy {Lambda_entropy} \
  --ckpt {Checkpoint} \
  --save_mesh \
  --max_steps {Max_steps}

# Testing

In [ ]:
%cd /content/stable-dreamfusion

!python main.py \
  -O \
  --test \
  --workspace {Workspace_test} \
  --save_mesh

# Show result

In [ ]:
def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

In [ ]:
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))

In [ ]:
# show video
clip = VideoFileClip(rgb_video)
clip = resize(clip, height=420)
clip.ipython_display()